In [1]:
import ArchGDAL

In [2]:
function geo2pixel(gt, x, y)
	Int64(round((x-gt[4])/gt[6])), Int64(round((y-gt[1])/gt[2]))
end
function getelev(x, y)
	ArchGDAL.registerdrivers() do
	    ArchGDAL.read("srtm_21_05.tif") do dataset
	        band = ArchGDAL.getband(dataset, 1)
	        geotransform = ArchGDAL.getgeotransform(dataset)
	        arr = ArchGDAL.read(band)
	        newcoords = geo2pixel(geotransform, x, y)
	        return arr[newcoords[2], newcoords[1]]
	    end
	end
end

getelev (generic function with 1 method)

In [3]:
getelev(38.735375,-77.410316)

93

In [4]:
using HTTP
using JSON
using BenchmarkTools

In [9]:

randomcoord = (40-rand()*5, -80-rand()*-5)
print(randomcoord)
rstring = string("https://maps.googleapis.com/maps/api/elevation/json?locations=",string(randomcoord[1]),",",string(randomcoord[2]))
print(rstring)
@btime r = HTTP.request("GET", rstring)
@btime g_val = Float(JSON.parse(String(r.body))["results"][1]["elevation"])
@btime m_val = getelev(randomcoord[1], randomcoord[2])


(36.41356833170339, -77.56555114382891)https://maps.googleapis.com/maps/api/elevation/json?locations=36.41356833170339,-77.56555114382891

HTTP.ExceptionRequest.StatusError: HTTP.ExceptionRequest.StatusError(403, HTTP.Messages.Response:
"""
HTTP/1.1 403 Forbidden
Content-Length: 1103
Content-Type: text/html; charset=UTF-8
Date: Mon, 17 Sep 2018 18:42:21 GMT
Alt-Svc: quic=":443"; ma=2592000; v="44,43,39,35"

<html><head><meta http-equiv="content-type" content="text/html; charset=utf-8"/><title>Sorry...</title><style> body { font-family: verdana, arial, sans-serif; background-color: #fff; color: #000; }</style></head><body><div><table><tr><td><b><font face=sans-serif size=10><font color=#4285f4>G</font><font color=#ea4335>o</font><font color=#fbbc05>o</font><font color=#4285f4>g</font><font color=#34a853>l</font><font color=#ea4335>e</font></font></b></td><td style="text-align: left; vertical-align: bottom; padding-bottom: 15px; width: 50%"><div style="border-bottom: 1px solid #dfdfdf;">Sorry...</div></td></tr></table></div><div style="margin-left: 4em;"><h1>We're sorry...</h1><p>... but your computer or network may b

0.6598921179538282